# WIP

```{cypher}
CALL gds.graph.project(
  'testPRGraph',
  {
    Artist: {},
    Tag: {},
    Track: {}
  },
  {
    COLLAB_WITH: { properties: 'pg_weight' },
    HAS_TAG: { properties: 'pg_weight' },
    LINKED_TO: { properties: 'pg_weight' },
    MUSICALLY_RELATED_TO: { properties: 'pg_weight' },
    PERSONALLY_RELATED_TO: { properties: 'pg_weight' },
    TAGS: { properties: 'pg_weight' },
    WORKED_BY: { properties: 'pg_weight' },
    WORKED_IN: { properties: 'pg_weight' }
  }
);
```

AND

```{cypher}
CALL gds.pageRank.stream(
    'testPRGraph',
    {
        nodeLabels: ['Artist', 'Tag', 'Track'],
        relationshipTypes: ['COLLAB_WITH', 'HAS_TAG', 'LINKED_TO', 'MUSICALLY_RELATED_TO', 'PERSONALLY_RELATED_TO', 'TAGS', 'WORKED_BY', 'WORKED_IN'],
        concurrency: 4,
        logProgress: true,
        dampingFactor: 0.85,
        maxIterations: 20,
        tolerance: 0.0000000001,
        relationshipWeightProperty: 'pg_weight'
    }
);
```

is too slow.

TODO: Try with nx

In [1]:
from typing import Any
from neo4j import GraphDatabase, basic_auth, Driver
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import os
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

In [2]:
load_dotenv()

True

In [3]:
DB_HOST = os.getenv("NEO4J_HOST")
DB_PORT = os.getenv("NEO4J_PORT")
DB_USER = os.getenv("NEO4J_USER")
DB_PASS = os.getenv("NEO4J_PASS")

# .env validation
assert DB_HOST is not None and \
    DB_PORT is not None and \
    DB_USER is not None and \
    DB_PASS is not None, \
    "INVALID .env"

In [4]:
driver = GraphDatabase.driver(f"bolt://{DB_HOST}:{DB_PORT}", auth=basic_auth(DB_USER, DB_PASS))
driver

In [5]:
def execute_query_return(driver: Driver, query: str, params: None | dict[str, Any] = None) -> list[dict[str, Any]]:
    with driver.session() as session:
        result = session.run(query, params)  # type: ignore
        return result.data()

In [6]:
query = """
match (n:Artist {in_last_fm: true})
return n.main_id as main_id, n.listeners as listeners, n.playcount as playcount;
"""

data = pd.DataFrame(execute_query_return(driver, query))
len(data)

1373228

In [7]:
data.corr().loc["listeners", "playcount"]

0.7160603315140386

In [8]:
mmscaler = MinMaxScaler()

data["listeners_scaled"] = mmscaler.fit_transform(data[["listeners"]])
data["playcount_scaled"] = mmscaler.fit_transform(data[["playcount"]])
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00


In [9]:
data.head()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled
0,1889201,284,659,2.685916e-05,2.452997e-07
1,1037506,4,5,3.782980e-07,1.861151e-09
2,438424,1,1,9.457451e-08,3.722301e-10
3,1209708,812,6060,7.679450e-05,2.255715e-06
4,2557269,103,417,9.741174e-06,1.552200e-07


In [10]:
pca_calculator = PCA(1)
pca = pca_calculator.fit_transform(data[["listeners_scaled", "playcount_scaled"]])

In [11]:
pca_calculator.explained_variance_ratio_  # This is great

array([0.97250321])

In [12]:
pca_calculator.components_

array([[0.98357992, 0.18047308]])

In [13]:
pca

array([[-0.00149563],
       [-0.00152172],
       [-0.001522  ],
       ...,
       [ 0.0025691 ],
       [-0.00151985],
       [-0.00148763]])

In [14]:
data["popularity"] = pca
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,2.711631e-19
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00


In [15]:
data["popularity_scaled"] = mmscaler.fit_transform(data[["popularity"]])
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,2.711631e-19,1.512979e-03
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02,1.229366e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03,0.000000e+00
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03,2.777946e-06
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03,2.180286e-05
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03,1.983562e-04
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00,1.000000e+00


In [18]:
graph = nx.Graph()

# Nodes
query = "match (n:Artist {in_last_fm: true}) return n.main_id as main_id;"
with driver.session() as session:
    records = session.run(query)
    for i, artist in enumerate(records):
        graph.add_node(artist["main_id"])

        if i % 1000 == 0:
            print(i)

print("NODES DONE")

# Edges

query = """
    match (a:Artist {in_last_fm: true})-[r:COLLAB_WITH]->(b:Artist  {in_last_fm: true})
    where a < b
    return a.main_id as a, b.main_id as b, r.count as weight
    ;
"""

with driver.session() as session:
    records = session.run(query)
    for i, rel in enumerate(records):
        a, b, w = rel["a"], rel["b"], rel["weight"]

        graph.add_edge(a, b, weight=float(w))

        if i % 1000 == 0:
            print(i)

print("DONE!")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [19]:
pg = nx.pagerank(graph)
len(pg)

1373228

In [20]:
len(data)

1373228

In [23]:
pg.get(data.loc[0, "main_id"])

1.1225073473866158e-06

In [24]:
data["pagerank"] = data["main_id"].map(pg)
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,2.711631e-19,1.512979e-03,7.282112e-07
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02,1.229366e-02,9.070772e-07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03,0.000000e+00,5.035278e-07
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03,2.777946e-06,5.035278e-07
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03,2.180286e-05,5.035278e-07
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03,1.983562e-04,6.273237e-07
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00,1.000000e+00,4.541742e-04


In [31]:
data.sort_values(by="pagerank", inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
0,354885,0,0,0.000000,0.000000e+00,-0.001522,0.000000,5.035278e-07,5.035278e-07
1,475549,367,7686,0.000035,2.860961e-06,-0.001487,0.000034,5.035278e-07,4.820306e-01
2,1430415,372,769,0.000035,2.862450e-07,-0.001487,0.000034,5.035278e-07,4.820311e-01
3,1814908,370,3544,0.000035,1.319184e-06,-0.001487,0.000034,5.035278e-07,4.820316e-01
4,1520200,372,778,0.000035,2.895951e-07,-0.001487,0.000034,5.035278e-07,4.820327e-01


In [32]:
data.corr()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
main_id,1.000000,-0.039383,-0.013099,-0.039383,-0.013099,-0.038697,-0.038697,-0.084113,-0.193987
listeners,-0.039383,1.000000,0.716060,1.000000,0.716060,0.999524,0.999524,0.128396,0.231456
playcount,-0.013099,0.716060,1.000000,0.716060,1.000000,0.737246,0.737246,0.057557,0.098947
listeners_scaled,-0.039383,1.000000,0.716060,1.000000,0.716060,0.999524,0.999524,0.128396,0.231456
playcount_scaled,-0.013099,0.716060,1.000000,0.716060,1.000000,0.737246,0.737246,0.057557,0.098947
popularity,-0.038697,0.999524,0.737246,0.999524,0.737246,1.000000,1.000000,0.126816,0.228395
popularity_scaled,-0.038697,0.999524,0.737246,0.999524,0.737246,1.000000,1.000000,0.126816,0.228395
pagerank,-0.084113,0.128396,0.057557,0.128396,0.057557,0.126816,0.126816,1.000000,0.156502
pagerank_acc,-0.193987,0.231456,0.098947,0.231456,0.098947,0.228395,0.228395,0.156502,1.000000


In [33]:
data['pagerank_acc'] = data['pagerank'].cumsum()
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,1.034850e-19,1.512979e-03,7.282112e-07,3.665974e-01
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02,1.229366e-02,9.070772e-07,2.337343e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03,0.000000e+00,5.035278e-07,5.035278e-07
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03,2.777946e-06,5.035278e-07,1.728650e-01
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03,2.180286e-05,5.035278e-07,3.457295e-01
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03,1.983562e-04,6.273237e-07,5.254549e-01
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00,1.000000e+00,4.541742e-04,1.000000e+00


In [34]:
data.tail()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
1373223,97546,2774838,137106258,0.262429,0.051035,0.265808,0.265730,0.000102,0.999076
1373224,177,3979125,68405038,0.376324,0.025462,0.373218,0.372496,0.000111,0.999187
1373225,2327,2288231,87452010,0.216408,0.032552,0.217208,0.217420,0.000155,0.999342
1373226,762646,1403,13224,0.000133,0.000005,-0.001391,0.000131,0.000203,0.999546
1373227,491324,75375,8583151,0.007129,0.003195,0.006066,0.007543,0.000454,1.000000


In [35]:
data.pagerank_acc.corr(data.popularity_scaled)

0.06602290715838288

In [36]:
i = 0
current_cum_sum = 0
while i < len(data) - 1:
    if data.loc[i, "pagerank"] != data.loc[i + 1, "pagerank"]:
        current_cum_sum += data.loc[i, "pagerank"]
        data.loc[i, "pagerank_acc"] = current_cum_sum
        i += 1
    else:
        j = i + 1
        next_cum_sum = current_cum_sum + data.loc[i, "pagerank"]
        while data.loc[j, "pagerank"] == data.loc[i, "pagerank"] and  j < len(data) - 1:
            next_cum_sum += data.loc[j, "pagerank"]
            j += 1
        mean_cum_sum = (next_cum_sum + current_cum_sum) / 2.0
        for k in range(i, j):
            data.loc[k, "pagerank_acc"] = mean_cum_sum
        current_cum_sum = next_cum_sum
        i = j
    if i % 1000 == 0:
        print(i, current_cum_sum)
data.loc[i, "pagerank_acc"] = current_cum_sum + data.loc[i, "pagerank"]

876000 0.4410906321720644
877000 0.4415946892467438
878000 0.44209905334688665
879000 0.44260370867939736
881000 0.44361401223245034
883000 0.4446255021177708
885000 0.44563821817470983
889000 0.4476673263114526
890000 0.44817540361215064
891000 0.4486837944739712
895000 0.45072059417245675
896000 0.45123067048101423
899000 0.4527630692814479
904000 0.4553245031172863
908000 0.4573807821765897
909000 0.4578958876490572
917000 0.46203243506014086
927000 0.4672453079019474
931000 0.46934493480446043
936000 0.47198160625796554
940000 0.47410122588014453
944000 0.47623040059598737
949000 0.47890596645182887
952000 0.48051905419214036
956000 0.4826795101657202
958000 0.4837639586483977
962000 0.48594211823290634
971000 0.49089037610719993
974000 0.49255551563132477
981000 0.496472438431422
992000 0.5027279718122389
993000 0.5033020204611875
994000 0.5038780859170465
1000000 0.5073603730495234
1001000 0.5079438831662798
1009000 0.5126674103717668
1010000 0.513261873575704
1011000 0.513858820

In [37]:
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,1.034850e-19,1.512979e-03,7.282112e-07,3.665972e-01
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02,1.229366e-02,9.070772e-07,2.104982e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03,0.000000e+00,5.035278e-07,2.202330e-01
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03,2.777946e-06,5.035278e-07,2.202330e-01
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03,2.180286e-05,5.035278e-07,2.202330e-01
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03,1.983562e-04,6.273237e-07,5.255937e-01
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00,1.000000e+00,4.541742e-04,1.000000e+00


In [52]:
data.loc[int(0.75 * len(data)):, ["popularity_scaled", "pagerank_acc"]].corr()

,popularity_scaled,pagerank_acc
popularity_scaled,1.000000,0.113021
pagerank_acc,0.113021,1.000000


In [88]:
G_75 = graph.subgraph(data.loc[int(0.95 * len(data)):, "main_id"])
len(G_75)

68662

In [89]:
pg_75 = nx.pagerank(G_75)
data_75 = data.loc[int(0.95 * len(data)):].copy()
len(data_75) == len(pg_75)

True

In [90]:
data_75["pagerank"] = data_75["main_id"].map(pg_75)
data_75["pagerank_acc"] = data_75['pagerank'].cumsum()
data_75 = data_75.reset_index()
data_75.corr()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
index,1.000000,-0.148622,0.147310,0.064900,0.147310,0.064900,0.145567,0.145567,0.410247,0.981921
main_id,-0.148622,1.000000,-0.141598,-0.044941,-0.141598,-0.044941,-0.139076,-0.139076,-0.212269,-0.157599
listeners,0.147310,-0.141598,1.000000,0.671091,1.000000,0.671091,0.999353,0.999353,0.273220,0.169416
playcount,0.064900,-0.044941,0.671091,1.000000,0.671091,1.000000,0.697328,0.697328,0.124889,0.075244
listeners_scaled,0.147310,-0.141598,1.000000,0.671091,1.000000,0.671091,0.999353,0.999353,0.273220,0.169416
playcount_scaled,0.064900,-0.044941,0.671091,1.000000,0.671091,1.000000,0.697328,0.697328,0.124889,0.075244
popularity,0.145567,-0.139076,0.999353,0.697328,0.999353,0.697328,1.000000,1.000000,0.270206,0.167441
popularity_scaled,0.145567,-0.139076,0.999353,0.697328,0.999353,0.697328,1.000000,1.000000,0.270206,0.167441
pagerank,0.410247,-0.212269,0.273220,0.124889,0.273220,0.124889,0.270206,0.270206,1.000000,0.477258
pagerank_acc,0.981921,-0.157599,0.169416,0.075244,0.169416,0.075244,0.167441,0.167441,0.477258,1.000000


In [91]:
i = 0
current_cum_sum = 0
while i < len(data_75) - 1:
    if data_75.loc[i, "pagerank"] != data_75.loc[i + 1, "pagerank"]:
        current_cum_sum += data_75.loc[i, "pagerank"]
        data_75.loc[i, "pagerank_acc"] = current_cum_sum
        i += 1
    else:
        j = i + 1
        next_cum_sum = current_cum_sum + data_75.loc[i, "pagerank"]
        while data_75.loc[j, "pagerank"] == data_75.loc[i, "pagerank"] and  j < len(data_75) - 1:
            next_cum_sum += data_75.loc[j, "pagerank"]
            j += 1
        mean_cum_sum = (next_cum_sum + current_cum_sum) / 2.0
        for k in range(i, j):
            data_75.loc[k, "pagerank_acc"] = mean_cum_sum
        current_cum_sum = next_cum_sum
        i = j
    if i % 1000 == 0:
        print(i, current_cum_sum)
data_75.loc[i, "pagerank_acc"] = current_cum_sum + data_75.loc[i, "pagerank"]

1000 0.010163726687914433
2000 0.019754547124903498
3000 0.030270018138133778
4000 0.040931106931829137
5000 0.050242427060527464
6000 0.060729861632815914
7000 0.07210185218537679
8000 0.08277097563295432
9000 0.09424300496507071
15000 0.11707606302328229
16000 0.1257227389458758
17000 0.13473065491810182
18000 0.1441207376663311
19000 0.15408469609752803
20000 0.16427029670805035
21000 0.17397547895360405
22000 0.18424889780144324
23000 0.19404835359354244
24000 0.20535000647886117
25000 0.2151120988898597
26000 0.2263813577472505
27000 0.2378535529416095
28000 0.2500815875065571
29000 0.2583949718863279
30000 0.2697874872917138
31000 0.2813040824372115
32000 0.2928672481913519
33000 0.3048296883495154
34000 0.3166961720879189
35000 0.3292477833685206
38000 0.35849714532057675
39000 0.369869097071259
40000 0.3825833453823251
41000 0.3950581830445316
42000 0.4078928205820824
43000 0.42115191324962503
44000 0.4355196838553473
45000 0.447560470765862
46000 0.46120835112621467
47000 0.47

In [92]:
data_75.corr()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
index,1.000000,-0.148622,0.147310,0.064900,0.147310,0.064900,0.145567,0.145567,0.410247,0.981921
main_id,-0.148622,1.000000,-0.141598,-0.044941,-0.141598,-0.044941,-0.139076,-0.139076,-0.212269,-0.157599
listeners,0.147310,-0.141598,1.000000,0.671091,1.000000,0.671091,0.999353,0.999353,0.273220,0.169416
playcount,0.064900,-0.044941,0.671091,1.000000,0.671091,1.000000,0.697328,0.697328,0.124889,0.075244
listeners_scaled,0.147310,-0.141598,1.000000,0.671091,1.000000,0.671091,0.999353,0.999353,0.273220,0.169416
playcount_scaled,0.064900,-0.044941,0.671091,1.000000,0.671091,1.000000,0.697328,0.697328,0.124889,0.075244
popularity,0.145567,-0.139076,0.999353,0.697328,0.999353,0.697328,1.000000,1.000000,0.270206,0.167441
popularity_scaled,0.145567,-0.139076,0.999353,0.697328,0.999353,0.697328,1.000000,1.000000,0.270206,0.167441
pagerank,0.410247,-0.212269,0.273220,0.124889,0.273220,0.124889,0.270206,0.270206,1.000000,0.477258
pagerank_acc,0.981921,-0.157599,0.169416,0.075244,0.169416,0.075244,0.167441,0.167441,0.477258,1.000000


In [86]:
data_75.tail()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
68657,1373223,97546,2774838,137106258,0.262429,0.051035,0.265808,0.265730,0.000267,0.998521
68658,1373224,177,3979125,68405038,0.376324,0.025462,0.373218,0.372496,0.000599,0.999125
68659,1373225,2327,2288231,87452010,0.216408,0.032552,0.217208,0.217420,0.000561,0.999690
68660,1373226,762646,1403,13224,0.000133,0.000005,-0.001391,0.000131,0.000307,1.000000
68661,1373227,491324,75375,8583151,0.007129,0.003195,0.006066,0.007543,0.000930,1.000937


In [35]:
driver.close()